In [1]:
!pip install apache_beam
!pip install datasets

     |████████████████████████████████| 8.9MB 10.8MB/s 
     |████████████████████████████████| 63.8MB 46kB/s 
     |████████████████████████████████| 829kB 46.9MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 153kB 56.9MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 2.0MB 51.0MB/s 
     |████████████████████████████████| 112kB 57.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=e959d692b4f20048860d0cfaee2803eb2671aa2c1a0463804f65553d013f63b4
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-cp36-none-any.whl size=43515 sha256=d91691c7976b4d708d006b9bcc40f96d4f433c47556fcaf1ebd371a030e1e099
  Stored in directory: /root/.cache/pip/wheels/5a/d3/be/86620c9dd3fca68986c33

In [1]:
from datasets import list_datasets, list_metrics, load_dataset, load_metric
from pprint import pprint
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
import numpy as np

import random
import math
import time

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
datasets = list_datasets()

In [6]:
print(f"🤩 Currently {len(datasets)} datasets are available on the hub:")
pprint(datasets, compact=True)

🤩 Currently 601 datasets are available on the hub:
['acronym_identification', 'ade_corpus_v2', 'aeslc', 'afrikaans_ner_corpus',
 'ag_news', 'ai2_arc', 'air_dialogue', 'ajgt_twitter_ar', 'allegro_reviews',
 'allocine', 'alt', 'amazon_polarity', 'amazon_reviews_multi',
 'amazon_us_reviews', 'ambig_qa', 'amttl', 'anli', 'app_reviews', 'aqua_rat',
 'aquamuse', 'ar_cov19', 'ar_res_reviews', 'arabic_billion_words',
 'arabic_pos_dialect', 'arcd', 'arsentd_lev', 'art', 'arxiv_dataset',
 'aslg_pc12', 'asnq', 'asset', 'assin', 'assin2', 'autshumato', 'bc2gm_corpus',
 'best2009', 'bianet', 'bible_para', 'big_patent', 'billsum',
 'bing_coronavirus_query_set', 'biomrc', 'blended_skill_talk', 'blimp',
 'blog_authorship_corpus', 'bookcorpus', 'bookcorpusopen', 'boolq', 'bprec',
 'break_data', 'brwac', 'bsd_ja_en', 'bswac', 'c3', 'c4', 'cail2018', 'capes',
 'catalonia_independence', 'cawac', 'cc100', 'cdsc', 'cdt', 'cfq', 'chr_en',
 'cifar10', 'circa', 'civil_comments', 'clickbait_news_bg', 'climate_f

In [7]:
squad = list_datasets(with_details=True)[datasets.index('squad')]
pprint(squad.__dict__) 

{'author': None,
 'citation': '@article{2016arXiv160605250R,\n'
             '       author = {{Rajpurkar}, Pranav and {Zhang}, Jian and '
             '{Lopyrev},\n'
             '                 Konstantin and {Liang}, Percy},\n'
             '        title = "{SQuAD: 100,000+ Questions for Machine '
             'Comprehension of Text}",\n'
             '      journal = {arXiv e-prints},\n'
             '         year = 2016,\n'
             '          eid = {arXiv:1606.05250},\n'
             '        pages = {arXiv:1606.05250},\n'
             'archivePrefix = {arXiv},\n'
             '       eprint = {1606.05250},\n'
             '}',
 'description': 'Stanford Question Answering Dataset (SQuAD) is a reading '
                'comprehension dataset, consisting of questions posed by '
                'crowdworkers on a set of Wikipedia articles, where the answer '
                'to every question is a segment of text, or span, from the '
                'corresponding reading pa

In [8]:
dataset = load_dataset('squad')

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/1244d044b266a5e4dbd4174d23cb995eead372fbca31a03edc3f8a132787af41. Subsequent calls will reuse this data.


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [10]:
train_context, train_question = dataset['train'].__getitem__('context'), dataset['train'].__getitem__('question')
train_ans = dataset['train'].__getitem__('answers')

val_context, val_question = dataset['validation'].__getitem__('context'), dataset['validation'].__getitem__('question')
val_ans = dataset['validation'].__getitem__('answers')

In [11]:
train_ans[10]

{'answer_start': [119], 'text': ['Rome']}

In [12]:
Train = [[train_context[i] + ' ' +  train_question[i], train_ans[i]['text'][0]] for i in range(len(train_question))]
Val = [[val_context[i] + ' ' +  val_question[i], val_ans[i]['text'][0] ] for i in range(len(val_question))]

train_df = pd.DataFrame(Train, columns = ['QC','Answers'])
train_df.to_csv('Train.csv', index =  False)
val_df = pd.DataFrame(Val, columns = ['QC','Answers'])
val_df.to_csv('Validation.csv', index = False)

In [13]:
spacy_en = spacy.load('en')
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [14]:
SRC = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

In [15]:
train_data, val_data = TabularDataset.splits(path='.',
                                             train = 'Train.csv',
                                             validation = 'Validation.csv',
                                             skip_header = True,
                                             format = 'csv',
                                             fields =[('src',SRC), ('trg', TRG)])

In [36]:
vars(val_data.examples[0])

{'src': ['super',
  'bowl',
  '50',
  'was',
  'an',
  'american',
  'football',
  'game',
  'to',
  'determine',
  'the',
  'champion',
  'of',
  'the',
  'national',
  'football',
  'league',
  '(',
  'nfl',
  ')',
  'for',
  'the',
  '2015',
  'season',
  '.',
  'the',
  'american',
  'football',
  'conference',
  '(',
  'afc',
  ')',
  'champion',
  'denver',
  'broncos',
  'defeated',
  'the',
  'national',
  'football',
  'conference',
  '(',
  'nfc',
  ')',
  'champion',
  'carolina',
  'panthers',
  '24–10',
  'to',
  'earn',
  'their',
  'third',
  'super',
  'bowl',
  'title',
  '.',
  'the',
  'game',
  'was',
  'played',
  'on',
  'february',
  '7',
  ',',
  '2016',
  ',',
  'at',
  'levi',
  "'s",
  'stadium',
  'in',
  'the',
  'san',
  'francisco',
  'bay',
  'area',
  'at',
  'santa',
  'clara',
  ',',
  'california',
  '.',
  'as',
  'this',
  'was',
  'the',
  '50th',
  'super',
  'bowl',
  ',',
  'the',
  'league',
  'emphasized',
  'the',
  '"',
  'golden',
  'anniv

In [17]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [38]:
BATCH_SIZE = 256

train_iterator, val_iterator = BucketIterator.splits(
    (train_data, val_data), 
    batch_size = BATCH_SIZE, 
    sort = False,
    device = device)

In [39]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [40]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [41]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [42]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [44]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(87805, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(19331, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=19331, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [45]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 55,341,699 trainable parameters


In [46]:
optimizer = optim.Adam(model.parameters())

In [47]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [48]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [49]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [50]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 5m 55s
	Train Loss: 6.371 | Train PPL: 584.911
	 Val. Loss: 5.726 |  Val. PPL: 306.619
